In [8]:
##### Epoching

import glob
import os

import numpy as np

import mne


In [9]:
# Assign two lists of important codes to experimental stimuli
B_codes = {71,72,73,74,75} 
Y_codes = {77,78,79,80,81}

In [13]:
%%capture
path = './raw/'
for file in glob.glob(os.path.join(path, '*_ICA-raw.fif')):
    
    filepath, filename = os.path.split(file)
    filename, ext = os.path.splitext(filename)    
    
    # Load the pre-processed raw data
    raw = mne.io.read_raw_fif(file, preload=True) 
    picks = mne.pick_types(raw.info, eeg=True, eog=False)
    
    # Find the events (three columns array with time sample,
    # previous event code, and current event code as columns)
    events = mne.find_events(raw, initial_event=True) 
    # Look for the break codes between blocks
    breaks = np.where(events[:,2]==245)[0]
    
    # Cut the data after the inital training phase
    raw.crop(tmin=int(events[:,0][breaks[1]]/1024))
    events = mne.find_events(raw, initial_event=True)
    # Get the new time samples for the remaining breaks
    breaks = np.where(events[:,2]==245)[0]
    
    # Divide the data into a baseline and a dual regulation set
    raw_base = raw.copy().crop(tmin=int(raw.times.min()),
                               tmax=int(events[:,0][breaks[1]]/1024))
    events_base = mne.find_events(raw_base, initial_event=True)

    raw_reg = raw.copy().crop(tmin=int(events[:,0][breaks[1]]/1024),
                              tmax=int(raw.times.max()))
    events_reg = mne.find_events(raw_reg, initial_event=True)
    
    # Re-coding events
    # Create a 1D array with all codes.
    new_events_base = events_base[:,2]
    
    # Create a intermediary variable for noting the cue code and set it to 0.
    # temp_cue will be used to store information about the first stimulus
    # in our sequence, which is 'A' (1) or 'B' (2).
    temp_cue = 0
    
    for i in range(new_events_base[:-3].size):
        
        # The first if statement codes the cue. If an 'A' is found and the temp_cue is 0.
        # set the temp_cue to 1 (i.e., we found an A).
        if (new_events_base[i] == 70 and temp_cue == 0 
            and any(x in (12, 13) for x in new_events_base[range(i,i+3)])):
            temp_cue = 1
            new_events_base[i] = 5
        # Otherwise, if any of the codes for 'B' is found and the temp_cue is 0, set the
        # temp_cue to 2.
        elif (new_events_base[i] in B_codes and temp_cue == 0 
              and any(x in (12, 13) for x in new_events_base[range(i,i+3)])):
            temp_cue = 2
            new_events_base[i] = 6
        
        # After we coded the cue-section of the sequence, we move on to the probe.
        if temp_cue == 1:
            # If the temp_cue has already been set through the first if condition, is set 1,
            # and a code for 'X' is found, change the value in row i, which corresponds to the
            # current probe value to a new value that specifically codes 'AX'.
            if (new_events_base[i] == 76 
                and any(x in (12, 13) for x in new_events_base[range(i,i+2)])):
                new_events_base[i] = 1
                # Don't forget to set the temp_cue back to 0.
                temp_cue = 0
            # Now do the same for 'AY' with one statement for each code of 'Y'.
            elif (new_events_base[i] in Y_codes
                  and any(x in (12, 13) for x in new_events_base[range(i,i+2)])):
                new_events_base[i] = 3
                temp_cue = 0
        elif temp_cue == 2:
            if (new_events_base[i] == 76
                and any(x in (12, 13) for x in new_events_base[range(i,i+2)])):
                new_events_base[i] = 2
                temp_cue = 0
            elif (new_events_base[i] in Y_codes
                  and any(x in (12, 13) for x in new_events_base[range(i,i+2)])):
                new_events_base[i] = 4
                temp_cue = 0
    
    events_base[:,2] = new_events_base
    
    # Re-coding events
    # Create a 1D array with all codes.
    new_events_reg = events_reg[:,2]
    
    # Create a intermediary variable for noting the cue code and set it to 0.
    # temp_cue will be used to store information about the first stimulus
    # in our sequence, which is 'A' (1) or 'B' (2).
    temp_cue = 0
    
    for i in range(new_events_reg[:-3].size):
        
        # The first if statement codes the cue. If an 'A' is found and the temp_cue is 0.
        # set the temp_cue to 1 (i.e., we found an A).
        if (new_events_reg[i] == 70 and temp_cue == 0 
            and any(x in (12, 13) for x in new_events_reg[range(i,i+3)])):
            temp_cue = 1
            new_events_reg[i] = 25
        # Otherwise, if any of the codes for 'B' is found and the temp_cue is 0, set the
        # temp_cue to 2.
        elif (new_events_reg[i] in B_codes and temp_cue == 0 
              and any(x in (12, 13) for x in new_events_reg[range(i,i+3)])):
            temp_cue = 2
            new_events_reg[i] = 26

        # After we coded the cue-section of the sequence, we move on to the probe.
        if temp_cue == 1:
            # If the temp_cue has already been set through the first if condition, is set 1,
            # and a code for 'X' is found, change the value in row i, which corresponds to the
            # current probe value to a new value that specifically codes 'AX'.
            if (new_events_reg[i] == 76 
                and any(x in (12, 13) for x in new_events_reg[range(i,i+2)])):
                new_events_reg[i] = 21
                # Don't forget to set the temp_cue back to 0.
                temp_cue = 0
            # Now do the same for 'AY' with one statement for each code of 'Y'.
            elif (new_events_reg[i] in Y_codes
                  and any(x in (12, 13) for x in new_events_reg[range(i,i+2)])):
                new_events_reg[i] = 23
                temp_cue = 0
        elif temp_cue == 2:
            if (new_events_reg[i] == 76
                and any(x in (12, 13) for x in new_events_reg[range(i,i+2)])):
                new_events_reg[i] = 22
                temp_cue = 0
            elif (new_events_reg[i] in Y_codes
                  and any(x in (12, 13) for x in new_events_reg[range(i,i+2)])):
                new_events_reg[i] = 24
                temp_cue = 0  
                
    # After re-writing the event codes, replace the old event column
    # with the new one
    events_reg[:,2] = new_events_reg
    
    # Write the event ids
    event_id_base = {'AX/base': 1, 'BX/base': 2, 'AY/base': 3, 'BY/base': 4, 'A/base': 5, 'B/base': 6}
    event_id_reg = {'AX/reg': 21, 'BX/reg': 22, 'AY/reg': 23, 'BY/reg': 24, 'A/reg': 25, 'B/reg': 26}
    
    # Epoch the two sets of continuous signal
    epochs_base = mne.Epochs(raw_base, events=events_base, event_id=event_id_base, tmin=-2, tmax=2.5,
                             baseline=None, picks=picks, preload=False)
    epochs_reg = mne.Epochs(raw_reg, events=events_reg, event_id=event_id_reg, tmin=-2, tmax=2.5,
                            baseline=None, picks=picks, preload=False)
    
    epochs = mne.concatenate_epochs([epochs_base, epochs_reg]) 
    
    # Save the epoched data
    epochs_base.save('./epochs/' + filename[:-8] + '_base-epo.fif')
    epochs_reg.save('./epochs/' + filename[:-8] + '_reg-epo.fif')
    epochs.save('./epochs/' + filename[:-8] + '_base_reg-epo.fif')